In [22]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

In [23]:
img = cv2.imread('red.png')

In [24]:
resized_img = cv2.resize(img, (600, 800))
cv2.imshow('Image', resized_img) #resizing image to fit the window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
lower_cone = np.array([0, 80, 120])
upper_cone = np.array([20, 255, 255]) #finding the lower and upper bounds of color range of cone

In [26]:
hsv = cv2.cvtColor(resized_img, cv2.COLOR_BGR2HSV)

In [27]:
mask = cv2.inRange(hsv, lower_cone, upper_cone)

In [28]:
cv2.imshow('Image2', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
img_thresh_low = cv2.inRange(hsv, np.array([0, 135, 135]), np.array([15, 255, 255]))
img_thresh_high = cv2.inRange(hsv, np.array([159, 135, 135]), np.array([179, 255, 255]))

In [30]:
result_img = cv2.bitwise_or(img_thresh_low, img_thresh_high)

In [31]:
cv2.imshow('Image3', result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
contours, hierarchy = cv2.findContours(result_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours onto the original image
filtered_contours_img = resized_img.copy()
line_img = np.zeros_like(filtered_contours_img)
min_area = 20
contour_list = []
contour_list_x_y = []
for contour in contours:
    area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)
    if h/w < 3 and h/w > 1.17 and area > min_area: #using height to width ratio
        #cv2.drawContours(filtered_contours_img, [contour], -1, (0, 255, 0), 2)   
        contour_list.append(contour)
        point=(x,y)
        contour_list_x_y.append(point) #adding x and y coordinates
        #print(contour_list)
        
contour_even = []
contour_even_x_y = []
for i in contour_list_x_y[::2]:
    contour_even_x_y.append(i)
contour_odd_x_y = set(contour_list_x_y) - set(contour_even_x_y)
contour_odd_x_y = list(contour_odd_x_y) #all the coordinates of the cones to the left
contour_even_x_y = list(contour_even_x_y)#all the coordinates of the cones to the right
slope1 = (contour_odd_x_y[-1][1] - contour_odd_x_y[0][1])/(contour_odd_x_y[-1][0]-contour_odd_x_y[0][0])
#finding slope of the endpoints of the left cones
pt = (contour_odd_x_y[0][0]-1000, int(slope1 * ((contour_odd_x_y[0][0]-1000) -contour_odd_x_y[0][0] ) + contour_odd_x_y[0][1]))

# calculate the endpoint of the line
x = pt[0] + 10000
y = int(slope1 * (x - pt[0]) + pt[1])
endpoint = (x, y)

cline=cv2.line(filtered_contours_img, pt, endpoint, (0, 0, 255), 2)
#finding slope of the cones to the right
slope2 = (contour_even_x_y[-1][1]-contour_even_x_y[0][1])/(contour_even_x_y[-1][0]-contour_even_x_y[0][0])
pt1 = (contour_even_x_y[0][0]-1000, int(slope2 * ((contour_even_x_y[0][0]-1000) -contour_even_x_y[0][0] ) + contour_even_x_y[0][1]))

# calculate the endpoint of the line
x1 = pt1[0] + 10000 #extending line to 10000 pixels
y1 = int(slope2 * (x1 - pt1[0]) + pt1[1])
endpoint1 = (x1, y1)

cline1=cv2.line(filtered_contours_img, pt1, endpoint1, (0, 0, 255), 2)

In [33]:
#Display the result
cv2.imshow('Contours', cline1)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("answer.png",cline1)

True